In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import rerun as rr
import torch
from pytorch3d.io import load_objs_as_meshes
from pytorch3d.renderer import (
    MeshRenderer, MeshRasterizer, FoVPerspectiveCameras,
    RasterizationSettings, look_at_view_transform,
    PointLights
)

import rerun_util as ru


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
device = torch.device("cuda:0")
meshes = load_objs_as_meshes(['data/cow_mesh/cow.obj'], device=device)

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/jorge/.conda/envs/texture/lib/python3.10/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/jorge/.conda/envs/texture/lib/python3.10/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt


In [4]:
from pytorch3d.renderer import HardPhongShader

torch.no_grad()

# lights
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

# cameras
dist = 2.7
azim = np.random.randint(0, 360)
elev = np.random.randint(0, 360)
R, T = look_at_view_transform(dist=dist, azim=azim, elev=elev)

cameras = FoVPerspectiveCameras(device=device, R=R, T=T, fov=60)

ru.pt3d_FovCamera(cameras)

# raster settings
raster_settings = RasterizationSettings(
    image_size=300,
    blur_radius=0.0,
    faces_per_pixel=1,
)

rasterizer = MeshRasterizer(cameras=cameras, raster_settings=raster_settings)
shader = HardPhongShader(device=device, cameras=cameras, lights=lights)

renderer = MeshRenderer(rasterizer=rasterizer, shader=shader)

renders = renderer(meshes, cameras=cameras).cpu().numpy()
fragments = rasterizer(meshes)

render = renders[0]
depth = fragments.zbuf[0, ..., 0].cpu().numpy()

# VISUALIZATION:
rr.init(spawn=True, application_id='meshvis')

rr.log('/', rr.ViewCoordinates.LUF, static=True)
rr.log('/', rr.Transform3D())

# log mesh
rr.log("mesh", ru.pt3d_mesh(meshes))

# log camera
rr.log('cam', ru.pt3d_FovCamera(cameras))
cam_trans = cameras.get_world_to_view_transform().inverse()
rr.log('cam', ru.pt3d_transform(cam_trans))

print(depth.shape)
# log depth
rr.log('cam', rr.Image(depth))


(300, 300)


[2024-08-22T15:20:37Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-08-22T15:20:37Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:56960
[2024-08-22T15:20:37Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1
